In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
sns.set(style="whitegrid", palette="viridis")

df = pd.read_csv("../data/raw/spain_energy.csv")

df.head()

,datetime,id,name,geoid,geoname,value
0,2014-01-01 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,25.280833
1,2014-01-02 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,39.924167
2,2014-01-03 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,4.992083
3,2014-01-04 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,4.091667
4,2014-01-05 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,13.587500


In [3]:
df.rename(columns={
    'datetime': 'fecha_hora',
    'id': 'id_registro',
    'name': 'nombre_variable',
    'geoid': 'id_geografico',
    'geoname': 'nombre_region',
    'value': 'precio_energia'
}, inplace=True)

df.head()

,fecha_hora,id_registro,nombre_variable,id_geografico,nombre_region,precio_energia
0,2014-01-01 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,25.280833
1,2014-01-02 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,39.924167
2,2014-01-03 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,4.992083
3,2014-01-04 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,4.091667
4,2014-01-05 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,13.587500


In [4]:
df.isnull().sum()

fecha_hora             0
id_registro            0
nombre_variable     5478
id_geografico      29256
nombre_region      29256
precio_energia         0
dtype: int64

In [ ]:
df['nombre_variable'].unique()

In [6]:
df['id_geografico'].unique()

array([ 3.,  2.,  1., nan])

In [8]:
df['nombre_region'].unique()

array(['España', 'Francia', 'Portugal', nan], dtype=object)